In [9]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams.update({
    'font.family': 'serif',  # 使用衬线字体
    'font.serif': ['Times New Roman'],  # 指定 Times New Roman 字体
    'font.size': 12,
    'text.usetex': False,
    'axes.labelsize': 13,
    'axes.titlesize': 15,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'legend.fontsize': 11,
    'legend.title_fontsize': 11,
    'savefig.bbox': 'tight',
})

sns.set_style("whitegrid")
flierprops = dict(marker='o', markersize=2, linestyle='none')

In [2]:
size_map = {
    "2160P": "3840x2160",
    "1080P": "1920x1080",
    "720P": "1280x720",
    "540P": "960x540",
    "360P": "640x360",
}
fig_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/figs"
table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"

In [3]:
def rm_outliers(df, cols, threshold=1.5, lower_only=True):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - (threshold * IQR)
        upper_bound = Q3 + (threshold * IQR)

        if lower_only:
            df = df[df[col] >= lower_bound]
        else:
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

#### 2. 预测的是 ssim

In [10]:
""" 把每个序列所有的 dataframe 叠在一起 """
import os
import pandas as pd
from tqdm import tqdm

seqs = os.listdir(f"{table_dir}/BDBR-fitOnly/rd-ssim")
convex_df = pd.DataFrame()

for seq in seqs:
    for f in tqdm(os.listdir(f"{table_dir}/BDBR-fitOnly/rd-ssim/{seq}")):
        convex_df = pd.concat([convex_df, pd.read_csv(f"{table_dir}/BDBR-fitOnly/rd-ssim/{seq}/{f}")], axis=0)
        
convex_df = convex_df.reset_index(drop=True)
display(convex_df)

convex_df.to_csv(f"{table_dir}/BDBR-fitOnly/convex_ssim.csv", index=False)

100%|██████████| 9/9 [00:00<00:00, 253.48it/s]


,seqName,sceneId,func,preset,size,qp,pred_ssim,log2ssim,ssim,pred_log2bitrate,log2bitrate,bitrate,pred_bitrate,convex
0,Lecture-5aee,17,quadratic2,medium,360P,52,0.925493,-0.116694,0.922299,3.621622,4.352709,20.4313,12.308836,pred
1,Lecture-5aee,17,quadratic2,medium,1080P,22,0.996206,-0.007319,0.994940,12.495681,12.751314,6894.9026,5775.303574,pred
2,Lecture-5aee,17,quadratic2,medium,540P,22,0.992122,-0.011167,0.992289,11.046069,11.674140,3267.8817,2114.452826,pred
3,Lecture-5aee,17,quadratic2,medium,360P,22,0.990550,-0.014823,0.989778,9.874572,10.727016,1694.9374,938.733411,pred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60279,Lecture-2ed0,6,linear,medium,1080P,27,0.999336,-0.000929,0.999356,8.110842,8.278200,310.4463,276.443771,fixed
60280,Lecture-2ed0,6,linear,medium,360P,27,0.999092,-0.001377,0.999046,6.425991,6.517740,91.6295,85.983703,fixed
60281,Lecture-2ed0,6,linear,medium,540P,27,0.999193,-0.001169,0.999190,6.987135,7.199487,146.9811,126.863622,fixed
60282,Lecture-2ed0,6,linear,medium,720P,27,0.999140,-0.001259,0.999128,7.555798,7.747359,214.8758,188.157629,fixed


In [11]:
""" 计算 BDBR -> 1) 相较于传统 CAE (会有 loss);  2) 相较于 fixed QP (会有 gain) """
from fastCAE.metrics.bjontegaard import BD_PSNR
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 8)

grouped = convex_df.groupby(["seqName", "sceneId", "func", "preset"])
rows = []
for i, ((seqName, sceneId, func, preset), group) in tqdm(enumerate(grouped)):
    pred_convex_df   = group[group["convex"] == "pred"].sort_values(by="bitrate").reset_index(drop=True)
    actual_convex_df = group[group["convex"] == "actual"].sort_values(by="bitrate").reset_index(drop=True)
    fixed_convex_df  = group[group["convex"] == "fixed"].sort_values(by="bitrate").reset_index(drop=True)
    
    bd_ssim_cae = BD_PSNR(
        actual_convex_df["bitrate"].tolist(), actual_convex_df["ssim"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["ssim"].tolist(), piecewise=0
    )
    bd_ssim_fix = BD_PSNR(
        fixed_convex_df["bitrate"].tolist(), fixed_convex_df["ssim"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["ssim"].tolist(), piecewise=0
    )
    rows.append([seqName, sceneId, func, preset, bd_ssim_cae, "cae"])
    rows.append([seqName, sceneId, func, preset, bd_ssim_fix, "fixed"])
    
all_bdbr = pd.DataFrame(rows, columns=["seqName", "sceneId", "func", "preset", "bd_ssim", "anchor"])
all_bdbr = rm_outliers(all_bdbr, cols=["bd_ssim"], threshold=0.75)
display(all_bdbr)
display(all_bdbr.describe())                  

all_bdbr.to_csv(f"{table_dir}/BDBR-fitOnly/all_rd_ssim.csv", index=False)

2682it [00:05, 475.63it/s]


,seqName,sceneId,func,preset,bd_ssim,anchor
1,Lecture-003a,0,linear,faster,0.002335,fixed
2,Lecture-003a,0,linear,medium,-0.004043,cae
3,Lecture-003a,0,linear,medium,0.001858,fixed
4,Lecture-003a,0,linear,slower,-0.003868,cae
...,...,...,...,...,...,...
5360,Lecture-7f7e,4,quadratic2,medium,-0.003384,cae
5361,Lecture-7f7e,4,quadratic2,medium,0.002068,fixed
5362,Lecture-7f7e,4,quadratic2,slower,-0.003350,cae
5363,Lecture-7f7e,4,quadratic2,slower,0.002131,fixed


,sceneId,bd_ssim
count,4828.000000,4828.000000
mean,8.164250,0.001174
std,9.264396,0.005099
min,0.000000,-0.004185
25%,1.000000,-0.000700
50%,4.000000,0.000186
75%,12.000000,0.002361
max,39.000000,0.157388


In [13]:
# 看一下不同 func 的效果
table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"
all_bdbr = pd.read_csv(f"{table_dir}/BDBR-fitOnly/all_rd_ssim.csv")

grouped = all_bdbr.groupby(["anchor", "func", "preset", "seqName"], as_index=False)

rows = []
for (anchor, func, preset, seqName), group in grouped:
    group = group.reset_index(drop=True)
    rows.append([anchor, func, preset, seqName, group["bd_ssim"].mean()])

avg_bdbr = pd.DataFrame(rows, columns=["anchor", "func", "preset", "seqName", "bd_ssim"])
avg_bdbr = rm_outliers(avg_bdbr, cols=["bd_ssim"], threshold=1.25).reset_index(drop=True)
display(avg_bdbr)

avg_bdbr.to_csv(f"{table_dir}/BDBR-fitOnly/avg_rd_ssim.csv", index=False)

,anchor,func,preset,seqName,bd_ssim
0,cae,linear,faster,Lecture-03bc,-0.000737
1,cae,linear,faster,Lecture-07e0,-0.003636
2,cae,linear,faster,Lecture-0c8a,-0.002656
3,cae,linear,faster,Lecture-0f64,-0.002818
...,...,...,...,...,...
782,fixed,quadratic2,slower,Lecture-74cf,0.003073
783,fixed,quadratic2,slower,Lecture-7a57,0.000645
784,fixed,quadratic2,slower,Lecture-7e40,0.001372
785,fixed,quadratic2,slower,Lecture-7f7e,0.005074


In [14]:
display(avg_bdbr[avg_bdbr["anchor"] == "cae"].describe())       # 相比 orig CAE
display(avg_bdbr[avg_bdbr["anchor"] == "fixed"].describe())     # 相比 fixed ladder

,bd_ssim
count,381.000000
mean,-0.001672
std,0.001121
min,-0.004173
25%,-0.002454
50%,-0.001614
75%,-0.000746
max,0.000587


,bd_ssim
count,405.000000
mean,0.002530
std,0.003179
min,-0.000337
25%,0.000784
50%,0.001663
75%,0.003073
max,0.025791
